In [1]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import svm

def build_confusion_matrix(dataset, model):
        X_train, X_test, y_train, y_test = train_test_split(dataset.iloc[:, :-1], dataset.iloc[:, -1], test_size=0.25)
        predictions = model.fit(X_train, y_train).predict(X_test)
        tn, fn, fp, tp = confusion_matrix(predictions, y_test.values).ravel()
        return {'true_negative': tn, 'false_negative': fn, 'false_positive': fp, 'true_positive': tp}

def sensitivity(cm):
    return cm['true_positive'] / (cm['true_positive'] + cm['false_negative'])

def specificity(cm):
    return cm['true_negative'] / ( cm['true_negative'] + cm['false_positive'] )

def false_positive_rate(cm):
    return (1.0 - (specificity(cm)) )

def generate_scores(dataset, model):
    specificity_scores = []
    sensitivity_scores = []

    for n in range(1000):
        cm = build_confusion_matrix(dataset, model)
        specificity_scores.append(specificity(cm))
        sensitivity_scores.append(sensitivity(cm))
    
    spec_avg = pd.Series(specificity_scores).mean()
    sens_avg = pd.Series(sensitivity_scores).mean()
    
    return {'specificity': spec_avg, 'sensitivity': sens_avg }

def summarise_test(test):
    x ='For {}, in model {}\nAverage NO SIGFALL accuracy is {}\nAverage SIGFALL accuracy is {}\n---\n'
    print(x.format(test['data_name'],test['model'],test['specificity'],test['sensitivity']))   

In [2]:
# Load data sets
inc_updrs_sigfall = pd.read_csv('./working_data/normalised_increase_updrs_sigfall.csv')
inc_updrs_sigfall_raw = pd.read_csv('./working_data/increase_updrs_sigfall_raw.csv')
delta_updrs_sigfall = pd.read_csv('./working_data/normalised_delta_updrs_sigfall.csv')
delta_updrs_sigfall_raw = pd.read_csv('./working_data/delta_updrs_sigfall_raw.csv')
updrs_future_sigfall = pd.read_csv('./working_data/normalised_updrs_future_sigfall.csv')
updrs_future_sigfall_raw = pd.read_csv('./working_data/updrs_future_sigfall_raw.csv')

datasets = [{'data':inc_updrs_sigfall, 'name':'inc_updrs_sigfall'},
            {'data':inc_updrs_sigfall_raw, 'name':'inc_updrs_sigfall_raw'},
            {'data':delta_updrs_sigfall , 'name':'delta_updrs_sigfall'},
            {'data':delta_updrs_sigfall_raw , 'name':'delta_updrs_sigfall_raw'},
            {'data':updrs_future_sigfall , 'name':'updrs_future_sigfall'},
            {'data':updrs_future_sigfall_raw , 'name':'updrs_future_sigfall_raw'}
           ]

In [5]:
models = [
    svm.SVC(kernel="linear", class_weight={1: 10})
]

tests = []
for m in models:
    for d in datasets:
        tests.append({
            'data_name': d['name'],
            'dataset': d['data'],
            'model': m
        })

for t in tests:
    s = generate_scores(dataset=t['dataset'], model=t['model'])
    t['specificity'] = s['specificity']
    t['sensitivity'] = s['sensitivity']
    summarise_test(t)

For inc_updrs_sigfall, in model SVC(class_weight={1: 10}, kernel='linear')
Average NO SIGFALL accuracy is 0.8015763959878929
Average SIGFALL accuracy is 0.4755059766437297
---

For inc_updrs_sigfall_raw, in model SVC(class_weight={1: 10}, kernel='linear')
Average NO SIGFALL accuracy is 0.8072290759709716
Average SIGFALL accuracy is 0.4719485082456315
---

For delta_updrs_sigfall, in model SVC(class_weight={1: 10}, kernel='linear')
Average NO SIGFALL accuracy is 0.7110294850733156
Average SIGFALL accuracy is 0.5697485022917004
---

For delta_updrs_sigfall_raw, in model SVC(class_weight={1: 10}, kernel='linear')
Average NO SIGFALL accuracy is 0.7086887991214463
Average SIGFALL accuracy is 0.5799886241800469
---

For updrs_future_sigfall, in model SVC(class_weight={1: 10}, kernel='linear')
Average NO SIGFALL accuracy is 0.719324587680992
Average SIGFALL accuracy is 0.5236352001428235
---

For updrs_future_sigfall_raw, in model SVC(class_weight={1: 10}, kernel='linear')
Average NO SIGFALL 